<a href="https://colab.research.google.com/github/mcampo26/map/blob/master/Project3eoutline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#import necessary packages
!apt-get -qq install python-cartopy python3-cartopy
import cartopy
!pip install pykrige
# Now let's import packages
import pandas as pd
import matplotlib.pyplot as plt
import pylab #using as a plotting library for spatial data, make contours
import numpy as np #gridded datasets and matrices; array package

# import cartopy, which we will use to map
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features


# import the UniversalKriging function from pykrige.uk 
from pykrige.uk import UniversalKriging



Selecting previously unselected package python-pkg-resources.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../0-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-pyshp.
Preparing to unpack .../1-python-pyshp_1.2.12+ds-1_all.deb ...
Unpacking python-pyshp (1.2.12+ds-1) ...
Selecting previously unselected package python-shapely.
Preparing to unpack .../2-python-shapely_1.6.4-1_amd64.deb ...
Unpacking python-shapely (1.6.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-cartopy:amd64.
Preparing to unpack .../4-python-cartopy_0.14.2+dfsg1-2build3_amd64.deb ...
Unpacking python-cartopy:amd64 (0.14.2+dfsg1-2build3) ...
Selecting previously unselected package python3-pkg-resources.
Preparing to unpack .../5-pyth

In [4]:
!pip install pyproj

     |████████████████████████████████| 10.4MB 4.4MB/s 


In [5]:
df = pd.read_excel('https://github.com/mcampo26/map/blob/master/FAKEpumpage.xlsx?raw=true') #read excel data
print(df)

    Pumpage 2018 (gallons/yr)  Pumpage 2019 (gallons/yr)    lam_x    lam_y
0                     4346400                       3750  3513064  3053266
1                     1932000                      93000  3513839  3050592
2                   372610000                     110000  3389816  3121532
3                    77163000                    3687600  3386729  3135944
4                    33228000                     198000  3506761  3068194
5                   380355961                  301516000  3506761  3066874
6                   346099000                     780000  3371537  3085976
7                   166628000                  106710000  3514499  3050592
8                   102275000                  317814000  3460783  3081317
9                   109078000                  212898000  3377663  3065950
10                  130566000                  200000000  3385049  3125841
11                  137053000                   85035000  3447901  3085064
12                   4954

In [0]:
illimap= {'proj': 'lcc', # Lambert Conformal Conic
     'ellps': 'clrk66', #Clarke 1866 ellipsoid
     'lon_0': -89.5, #Central Meridian
     'lat_0': 33, #Latitude of Origin
     'lat_1': 33, #Standard Parallel 1
     'lat_2': 45, #Standard Parallel 2
     'x_0': 2999994*0.3048006096012192, #starting x-coord. in ft, python expects m
     'y_0': 0} #starting y-coord.

wgs84= Proj('epsg:4326') #Standard lat/long coordinates

In [16]:
  from pyproj import Proj, transform
  inProj = Proj(illimap) #define input projection
  outProj = wgs84 #define output projection
  Long1,Lat1 = 0.3048006096012192*df['lam_x'].iloc[0],0.3048006096012192*df['lam_y'].iloc[0] #define input coordinates; convert lambert ft to m
  Long2,Lat2 = transform(inProj,outProj,Long1,Lat1) #convert from lambert to wgs84
print(Long2,Lat2)

41.40523623752517 -87.62095306984594


In [17]:
Latitude=[] 
Longitude=[]
for index, row in df.iterrows():
  from pyproj import Proj, transform
  inProj = Proj(illimap) #define input projection
  outProj = wgs84 #define output projection
  Long1,Lat1 = (0.3048006096012192*row['lam_x'],0.3048006096012192*row['lam_y']) #define input coordinates; convert lambert ft to m
  Long2,Lat2 = transform(inProj,outProj,Long1,Lat1) #convert from lambert to wgs84
  Latitude.append(Lat2) #add latitude data to empty Latitude=[] array
  Longitude.append(Long2) #add longitude data to empty Longitude=[] array
df['Latitude']=Latitude
df['Longitude']=Longitude
print(df)

    Pumpage 2018 (gallons/yr)  Pumpage 2019 (gallons/yr)  ...   Latitude  Longitude
0                     4346400                       3750  ... -87.620953  41.405236
1                     1932000                      93000  ... -87.618318  41.397821
2                   372610000                     110000  ... -88.068302  41.599605
3                    77163000                    3687600  ... -88.078809  41.639464
4                    33228000                     198000  ... -87.642913  41.446744
5                   380355961                  301516000  ... -87.643012  41.443106
6                   346099000                     780000  ... -88.137381  41.502368
7                   166628000                  106710000  ... -87.615902  41.397783
8                   102275000                  317814000  ... -87.810466  41.485393
9                   109078000                  212898000  ... -88.116037  41.446905
10                  130566000                  200000000  ... -88.085560  41